In [1]:
import pandas as pd

from task2.utils import count_unique_icgc_mutations, get_sorted_mutation_count, get_max_and_min_icgc_mutation_count

### Question 1: BASH

**Question 1.1**: *How to properly track error in your bash, when you call other Unix
commands, your own Python script, Perl script, or SQL statements via sqlplus (Oracle
command line client that can run a SQL file). Please specify for every case you know.*

- Redirect stderror to a file which can then be inspected: `python some_script.py 2>error.log.txt`
- If a program dumps everything to stdout, including error messages, then we probably need both stdout and stderr in a single text file: `./some_command.sh > all_output.log.txt 2>&1`
- If the error is known and we want to inspect it in CLI, we can pipe the output through grep: `./some_command.sh 2>&1 | grep -i -A 15 -B 15 “.*SomeError*”`

**Question 1.2**: *If your Bash script contains asynchronized commands, such as IBM LSF job
submission, how to compose your script to make sure everything works well and exits with a
proper code?*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If an asynchronous command can be run synchronously (i.e. wait for a job to complete), then several such processes can be run in the background and `wait` command can be used to wait for all of them to exit properly. Example:

```bash
#! /bin/zsh

python some_sync_script.py &
./async_script.sh --sync &
wait
```

**Question 1.3**: *Why the code comment is important in Bash? And how to give proper
comments in a Bash script?*

- Inline comments start with `#`. Multiline comments start with `: '` and end with `'`
- Comments can make a bash script easier to maintain by explaining the program's purpose and helping other developers to understand an algorithm if the logic flow is ambiguous.
- If the first comment is a shebang, e.g. `#! /bin/sh`, it tells the OS which interpreter to use (bash, zsh, Bourne shell)

Example:
```bash
#! /bin/zsh

: '
    some_script.sh accepts a series of integers to pass odd numbers to prog1.sh
    and even numbers to prog2.sh
'

for var in "$@"
do
    if ((var%2))
    then
        # var is an odd number
        # prog1 does not print even numbers, so its arguments should be odd numbers
       ./prog1.sh $var
    else
        # var is an even number
        ./prog2.sh $var
    fi
done
```


### Question 2: Data processing

#### Let's inspect the dataset first

In [2]:
data = pd.read_csv("simple_somatic_mutation.open.BLCA-CN.tsv", sep="\t")

In [3]:
data.columns

Index(['icgc_mutation_id', 'icgc_donor_id', 'project_code', 'icgc_specimen_id',
       'icgc_sample_id', 'matched_icgc_sample_id', 'submitted_sample_id',
       'submitted_matched_sample_id', 'chromosome', 'chromosome_start',
       'chromosome_end', 'chromosome_strand', 'assembly_version',
       'mutation_type', 'reference_genome_allele', 'mutated_from_allele',
       'mutated_to_allele', 'quality_score', 'probability', 'total_read_count',
       'mutant_allele_read_count', 'verification_status',
       'verification_platform', 'biological_validation_status',
       'biological_validation_platform', 'consequence_type', 'aa_mutation',
       'cds_mutation', 'gene_affected', 'transcript_affected',
       'gene_build_version', 'platform', 'experimental_protocol',
       'sequencing_strategy', 'base_calling_algorithm', 'alignment_algorithm',
       'variation_calling_algorithm', 'other_analysis_algorithm',
       'seq_coverage', 'raw_data_repository', 'raw_data_accession',
       'initia

In [4]:
data.shape

(124696, 42)

In [5]:
data.head(5)

,icgc_mutation_id,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,matched_icgc_sample_id,submitted_sample_id,submitted_matched_sample_id,chromosome,chromosome_start,...,experimental_protocol,sequencing_strategy,base_calling_algorithm,alignment_algorithm,variation_calling_algorithm,other_analysis_algorithm,seq_coverage,raw_data_repository,raw_data_accession,initial_data_release_date
0,MU5219,DO48399,BLCA-CN,SP106410,SA514847,SA514849,B54-Tumor,B54-Blood,3,178936091,...,Agilent SureSelect in Solution http://www.halo...,WXS,Illumina base-calling pipeline http://www.illu...,BWA http://bio-bwa.sourceforge.net/bwa.shtml,VarScan http://varscan.sourceforge.net/somatic...,Annotation with annovar http://www.openbioinfo...,70.0,EGA,EGAS00001000677,NaN
1,MU5219,DO48399,BLCA-CN,SP106410,SA514847,SA514849,B54-Tumor,B54-Blood,3,178936091,...,Agilent SureSelect in Solution http://www.halo...,WXS,Illumina base-calling pipeline http://www.illu...,BWA http://bio-bwa.sourceforge.net/bwa.shtml,VarScan http://varscan.sourceforge.net/somatic...,Annotation with annovar http://www.openbioinfo...,70.0,EGA,EGAS00001000677,NaN
2,MU4559679,DO48399,BLCA-CN,SP106410,SA514847,SA514849,B54-Tumor,B54-Blood,12,56558254,...,Agilent SureSelect in Solution http://www.halo...,WXS,Illumina base-calling pipeline http://www.illu...,BWA http://bio-bwa.sourceforge.net/bwa.shtml,VarScan http://varscan.sourceforge.net/somatic...,Annotation with annovar http://www.openbioinfo...,70.0,EGA,EGAS00001000677,NaN
3,MU4559679,DO48399,BLCA-CN,SP106410,SA514847,SA514849,B54-Tumor,B54-Blood,12,56558254,...,Agilent SureSelect in Solution http://www.halo...,WXS,Illumina base-calling pipeline http://www.illu...,BWA http://bio-bwa.sourceforge.net/bwa.shtml,VarScan http://varscan.sourceforge.net/somatic...,Annotation with annovar http://www.openbioinfo...,70.0,EGA,EGAS00001000677,NaN
4,MU4559679,DO48399,BLCA-CN,SP106410,SA514847,SA514849,B54-Tumor,B54-Blood,12,56558254,...,Agilent SureSelect in Solution http://www.halo...,WXS,Illumina base-calling pipeline http://www.illu...,BWA http://bio-bwa.sourceforge.net/bwa.shtml,VarScan http://varscan.sourceforge.net/somatic...,Annotation with annovar http://www.openbioinfo...,70.0,EGA,EGAS00001000677,NaN


**Question 2.1:** *Please can you demonstrate this task in python3. You may choose to use any
python module to parse your result, or write your code from scratch. For example, you can
use functions like df.groupy.*

*Based on file simple_somatic_mutation.open.BLCA-CN.tsv.gz, can you please print out all
the possible patterns of mutated_from_allele and mutated_to_allele. Then, please count how
many unique icgc_mutation_id are associated with those alleles changes.*

*Notes: Column transcripts_affected (alternative transcript) has duplicated the same
icgc_mutation_id multiple time for a single genomic location, therefore please only count the
unique_mutation_id once.*

In [6]:
count_unique_icgc_mutations(data)

,mutated_from_allele,mutated_to_allele,count_unique_icgc_mutation_ids
0,A,C,446
1,A,G,923
2,A,T,467
3,C,A,811
4,C,G,1404
5,C,T,3805
6,G,A,3620
7,G,C,1376
8,G,T,660
9,T,A,460


**Question 2.2:** *Please can you demonstrate this in python3.
Please find out which icgc_sample_id has the highest and lowest unique icgc_mutation_id
count.*

In [7]:
# All unique icgc_mutation_id counts per icgc_sample_id
get_sorted_mutation_count(data)

,icgc_sample_id,uniq_icgc_mutation_count
0,SA514800,583
1,SA514689,546
2,SA514687,480
3,SA514715,458
4,SA514791,424
...,...,...
98,SA514752,36
99,SA514863,36
100,SA514740,28
101,SA514880,27


In [8]:
# Maximum and minimum records should correspond to what we see in the cell above
max_rec, min_rec = get_max_and_min_icgc_mutation_count(data)
pd.DataFrame([max_rec, min_rec], index=["highest_mutation_count", "lowest_mutation_count"])

,icgc_sample_id,mutation_count
highest_mutation_count,SA514800,583
lowest_mutation_count,SA514876,14


**Question 2.3:** *Can you please create some tests in Python to check the functions previously
used in your code. For example the pytest package could be used.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Tests can be found in tests/test_task2.py*

### Question 3: Please answer each question and provide the SQL that you have used to get the result

**Question 3.1:** *How many genes in the gene table have an id_biotype of 23?*

```mysql
mysql> select count(*) from gene where id_biotype = 23;
+----------+
| count(*) |
+----------+
|      174 |
+----------+
1 row in set (0.00 sec)

```

**Question 3.2:** *What is the Ensembl Gene ID for the Gene_symbol TTTY2?*

```mysql
mysql> select ensembl_gene_id from gene where gene_symbol = 'TTTY2';
+-----------------+
| ensembl_gene_id |
+-----------------+
| ENSG00000212855 |
+-----------------+
1 row in set (0.00 sec)
```

**Question 3.3:** *Give a breakdown of the number of genes for each chromosome.*

```mysql
mysql> select chromosome, count(id_gene) from gene group by chromosome order by chromosome;
+------------+--------------------------+
| chromosome | count(distinct(id_gene)) |
+------------+--------------------------+
|          1 |                       51 |
|          2 |                       25 |
|          3 |                       31 |
|          4 |                       20 |
|          5 |                       25 |
|          6 |                       16 |
|          7 |                       19 |
|          8 |                       25 |
|          9 |                       18 |
|         10 |                       21 |
|         11 |                       25 |
|         12 |                       21 |
|         13 |                       16 |
|         14 |                       18 |
|         15 |                       17 |
|         16 |                       28 |
|         17 |                       27 |
|         18 |                        9 |
|         19 |                       27 |
|         20 |                       16 |
|         21 |                        9 |
|         22 |                       16 |
|         23 |                       16 |
|         24 |                        4 |
+------------+--------------------------+
24 rows in set (0.00 sec)
```


**Question 3.4:** *How many Transcripts does the Gene Symbol ﻿RAI14 has?*

```mysql
mysql> select count(t.id_transcript) from gene g join transcript t on g.id_gene = t.id_gene where g.gene_symbol = 'RAI14';
+------------------------+
| count(t.id_transcript) |
+------------------------+
|                     29 |
+------------------------+
1 row in set (0.00 sec)

```

**Question 3.5:** *What is the canonical transcript accession for Ensembl Gene id ﻿ENSG00000266960?*

```mysql
mysql> select g.ensembl_gene_id, t.accession from gene g join transcript t on g.id_gene = t.id_gene where g.ensembl_gene_id = 'ENSG00000266960';
+-----------------+-----------------+
| ensembl_gene_id | accession       |
+-----------------+-----------------+
| ENSG00000266960 | ENST00000586416 |
+-----------------+-----------------+
1 row in set (0.00 sec)
```

**Question 3.6:** *List the Transcript accessions for the Gene Symbol ﻿AK1 with id_biotype 23 and flags
gencode_basic*

```mysql
mysql> select g.gene_symbol, g.id_biotype, t.accession from gene g join transcript t on g.id_gene = t.id_gene where g.gene_symbol = 'AK1' and g.id_biotype=23;
+-------------+------------+-----------------+
| gene_symbol | id_biotype | accession       |
+-------------+------------+-----------------+
| AK1         |         23 | ENST00000550143 |
| AK1         |         23 | ENST00000413016 |
| AK1         |         23 | ENST00000223836 |
| AK1         |         23 | ENST00000550992 |
| AK1         |         23 | ENST00000373156 |
| AK1         |         23 | ENST00000373176 |
+-------------+------------+-----------------+
6 rows in set (0.00 sec)
```

**Question 3.7:** *Imagine that we have a table called “some_gene” with only a subset of the gene data. If I
want to join the gene table with this table but display all the genes in the result, what kind of
join would you do?*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If `gene` and `some_gene` have different sets of gene ids that might overlap and we need all gene ids from both tables in the result, then full outer join is needed.

**Question 3.8:** *Imagine that the gene and transcript tables are getting very big and that joining the two tables
get slower and slower. What would you do to improve performances?*

- Indexing `id_gene` columns in `gene` and `transcript` tables can speed up joins.
- Making `gene.id_gene` a primary key and adding FK constraint from `transcript.id_gene` to `gene.id_gene` would produce indexes that can speed up joins.
- Partitioning `gene` and `transcript` table on join key (`id_gene`) can speed up joins as well.

**Question 3.9:** *If you want to avoid duplicates in a table, what kind of index would you create?*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Primary keys and unique constraints

**Question 3.10:** *If you want to make sure that all the id_gene ids in the transcript table exists in the gene
table, what kind of index would you create?*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Make `gene.id_gene` a primary key (or unique) and add FK constraint from `transcript.id_gene` to `gene.id_gene`.

Final request:
- For Question 3.7, at first it wasn't very clear whether `some_gene` contained a subset of genes in `gene` table or all of genes `some_gene` belonged to a subset of genes in `gene` table. So, I provided an answer for a narrowed down scenario
- The set up was quite easy on macOS Monterey.